In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from time import sleep
import pandas as pd
from io import StringIO
from functools import partial
import matplotlib.pyplot as plt
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common import NoSuchElementException, StaleElementReferenceException
from matplotlib import style

print(plt.style.available)

['Solarize_Light2', '_classic_test_patch', '_mpl-gallery', '_mpl-gallery-nogrid', 'bmh', 'classic', 'dark_background', 'fast', 'fivethirtyeight', 'ggplot', 'grayscale', 'petroff10', 'seaborn-v0_8', 'seaborn-v0_8-bright', 'seaborn-v0_8-colorblind', 'seaborn-v0_8-dark', 'seaborn-v0_8-dark-palette', 'seaborn-v0_8-darkgrid', 'seaborn-v0_8-deep', 'seaborn-v0_8-muted', 'seaborn-v0_8-notebook', 'seaborn-v0_8-paper', 'seaborn-v0_8-pastel', 'seaborn-v0_8-poster', 'seaborn-v0_8-talk', 'seaborn-v0_8-ticks', 'seaborn-v0_8-white', 'seaborn-v0_8-whitegrid', 'tableau-colorblind10']


In [8]:
%matplotlib inline
try:
    # 尝试自动安装 ChromeDriver
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)
except Exception as e:
    print(f"自动安装失败: {e}")
    print("尝试手动指定 ChromeDriver...")
    
    # 手动指定 ChromeDriver 路径
    chrome_driver_path = "chromedriver.exe"  # 确保这个文件存在
    service = Service(chrome_driver_path)
    driver = webdriver.Chrome(service=service)

url='https://esf.fang.com/house-a015277-b05/'
driver.get(url)

In [9]:
import re
#截取面积数据的函数
def extract_area(text):
    area_match = re.search(r'(\d+\.?\d*)\s*㎡', text)
    if area_match:
        return area_match.group(1)
    else:
        return False

In [12]:
page=0
data=[]
while page<4:
    table=driver.find_element('class name','shop_list.shop_list_4')
    rows=table.find_elements(By.TAG_NAME,'dl')
    for row in rows:
        full_text=row.find_element('class name','tel_shop')
        area=extract_area(full_text.text)
        price_loc=row.find_element('class name','price_right')
        prices=price_loc.find_elements(By.TAG_NAME,'span')
        if area and prices:
            row_data=[area]+[price.text for price in prices]
            data.append(row_data)
    try:
        next_page=driver.find_element(By.LINK_TEXT,'下一页')
        next_page.click()
        page+=1
    except NoSuchElementException:
        break

In [13]:
df=pd.DataFrame(data,columns=['面积','价格','每平米价格'])

In [14]:
df

,面积,价格,每平米价格
0,81.07,598万,73763元/㎡
1,93.49,760万,81292元/㎡
2,33.15,286万,86274元/㎡
3,82.1,660万,80389元/㎡
4,93.3,900万,96463元/㎡
...,...,...,...
191,190.72,2360万,123742元/㎡
192,349.71,2930万,83784元/㎡
193,367.17,2600万,70812元/㎡
194,50.21,455万,90620元/㎡


In [15]:
df.to_csv('Beitaipingzhuang_Housing_Prices.csv',index=False,encoding='utf-8-sig')

In [25]:
try:
    # 尝试自动安装 ChromeDriver
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)
except Exception as e:
    print(f"自动安装失败: {e}")
    print("尝试手动指定 ChromeDriver...")
    
    # 手动指定 ChromeDriver 路径
    chrome_driver_path = "chromedriver.exe"  # 确保这个文件存在
    service = Service(chrome_driver_path)
    driver = webdriver.Chrome(service=service)

url='https://zu.fang.com/house-a015277-b05/'
driver.get(url)

In [50]:
page=0
data=[]
while page<5:
    table=driver.find_element('class name','houseList')
    rows=table.find_elements('class name','list.hiddenMap.rel')
    for row in rows:
        full_text=row.find_element('class name','font15.mt12.bold')
        area=extract_area(full_text.text)
        rent=row.find_element('class name','mt5.alingC').text
        if area and rent:
            row_data=[area,rent]
            data.append(row_data)
    try:
        next_page=driver.find_element(By.LINK_TEXT,'下一页')
        next_page.click()
        page+=1
    except NoSuchElementException:
        break
            
        

In [51]:
df=pd.DataFrame(data,columns=['面积','租价'])

In [52]:
df

,面积,租价
0,20,2550元/月
1,20,2650元/月
2,53,7400元/月
3,60,6300元/月
4,53,7100元/月
...,...,...
238,207,22000元/月
239,78,11000元/月
240,55,6500元/月
241,55,6500元/月


In [53]:
df.to_csv('Beitaipingzhuang_Rent.csv',index=False,encoding='utf-8-sig')